# LG Exaone Finetuning

Training 7B LG Exaone in bf16 format using AWS p4d and p4de clusters

**1-Library Installation and environment setting**

In [ ]:
!git config --global credential.helper store
!pip install huggingface_hub
!huggingface-cli login --token YOUR_HUGGINGFACE_KEYS
#YOUR_HUGGINGFACE_KEYS

In [ ]:
# Install AWS CLI if not already installed
!pip install awscli

In [ ]:
# Configure AWS CLI with your credentials
!aws configure

In [ ]:
!aws sts get-caller-identity


In [ ]:
!pip install --upgrade pip 
!pip install sagemaker transformers datasets peft trl bitsandbytes

In [ ]:
!pip install --upgrade sagemaker

**2-Define Training Datasets inside of your S3**

In [2]:
import os
import sagemaker
import boto3

# Specify your custom bucket name
bucket_name = "llama-training-s3"
region_name = sagemaker.Session().boto_region_name  # Detect the region
s3_prefix = "llama-training-s3"
train_s3_path = f"{s3_prefix}/train/exaone_train_set.tsv"
validation_s3_path = f"{s3_prefix}/validation/exaone_validation_set.tsv"
test_s3_path = f"{s3_prefix}/test/exaone_test_set.tsv"

[01/22/25 14:56:39] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=29211;file://C:\Users\javia\anaconda3\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=904193;file://C:\Users\javia\anaconda3\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

**3-Main Training File Creation**

Since all this is executed in a docker container and we cannot execute cell by cell, therefore, this code creates different codes in the current folder to be able to execute all of them inside of the AWS container to communicate with the cluster and the A100 8 GPUs

**Important Note:** Please, in this cell the main thing you need to change is the model name. It should correspond to the same model name you used for the data preprocessing. So for instance in the case of LG exaone, the huggingface name is: **LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct**. The second thing you will need to do is to ensure that the **.tsv** name matches the same as described in the previous cell. For example: **exaone_train_set.tsv**

**Secondary Note:** If you wonder why the preprocessing code is included inside of this code, there is a good reason. Since we cannot control the execution and ensure that tokenization was done right due to executing in a docker container, we preprocess inside of the container first to check all the tokenization works properly prior to passing it to the model. This way we can see what is happening inside of the docker container, otherwise, it would be very hard to debug since we cannot run cell by cell as in a normal notebook.

In [3]:
%%writefile train_deploy_huggingface.py
import os
import argparse
import subprocess
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    TrainerCallback,
)
from datasets import Dataset
import pandas as pd
import torch
import json
import deepspeed
import shutil 

def list_dir_contents(path):
    """
    Recursively lists all files and directories within the given path,
    along with their sizes.
    """
    print(f"\nContents of '{path}':")
    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        indent = ' ' * 4 * level
        print(f"{indent}{os.path.basename(root)}/")
        sub_indent = ' ' * 4 * (level + 1)
        for f in files:
            file_path = os.path.join(root, f)
            size = os.path.getsize(file_path) / 1e6  # Size in MB
            print(f"{sub_indent}{f} - {size:.2f} MB")

def main():
    parser = argparse.ArgumentParser()

    # Hyperparameters
    parser.add_argument("--batch_size", type=int, default=1, help="Per-device training batch size")
    parser.add_argument("--epochs", type=int, default=4, help="Number of training epochs")
    parser.add_argument("--learning_rate", type=float, default=1e-5, help="Learning rate")
    parser.add_argument("--gradient_accumulation_steps", type=int, default=16, help="Gradient accumulation steps")
    parser.add_argument("--max_length", type=int, default=4000, help="Maximum sequence length")

    # Environment variables set by SageMaker
    parser.add_argument("--model_dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--output_data_dir", type=str, default=os.environ.get("SM_OUTPUT_DATA_DIR"))
    parser.add_argument("--train_dir", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--validation_dir", type=str, default=os.environ.get("SM_CHANNEL_VALIDATION"))

    # **Add the following line to accept --local_rank**
    parser.add_argument("--local_rank", type=int, default=-1, help="Local rank for distributed training")

    args = parser.parse_args()

    #deepspeed.init_distributed()
    #device = torch.device(f"cuda:{args.local_rank}" if torch.cuda.is_available() else "cpu")
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct",
        #"meta-llama/Llama-3.1-8B-Instruct",
        #"meta-llama/Llama-3.2-3B-Instruct",
        use_fast=True,
        add_eos_token=True,
        add_bos_token=True,
        padding_side="left",
        trust_remote_code=True,
        token='YOUR_HUGGINGFACE_KEYS'  # Replace with your actual Hugging Face token
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct",
        #"meta-llama/Llama-3.1-8B-Instruct",
        #"meta-llama/Llama-3.2-3B-Instruct",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        token='YOUR_HUGGINGFACE_KEYS',  # Replace with your actual Hugging Face token
    )
    
    
    #model.to(device)
    #model.config.rope_scaling = {"type": "linear", "factor": 2.0}
    #model.config.use_cache = False
    model.gradient_checkpointing_enable()

    
    with open('./deepspeed_config.json', 'r') as config_file:
        deepspeed_config = json.load(config_file)
    


    # Move the model to the appropriate device
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #model.to(device)
    print(f"Model device: {next(model.parameters()).device}")
    
    
    # Prepare data
    def load_and_tokenize_data(input_path):
        df = pd.read_csv(input_path, sep='\t', header=None)
        texts = df[0].tolist()
        
        # Print the first sample for verification
        print("First sample before tokenization:")
        print(texts[0])
        
        tokenized_data = tokenizer(
            texts,
            truncation=True,
            max_length=args.max_length,
            padding="max_length",
        )
        
        # Print the tokenized version of the first sample
        print("\nTokenized input_ids of the first sample:")
        print(tokenized_data["input_ids"][0])
        
        # Optionally, decode the tokenized input_ids back to text for verification
        decoded_text = tokenizer.decode(tokenized_data["input_ids"][0])
        print("\nDecoded text from tokenized input_ids:")
        print(decoded_text)
        
        tokenized_data["labels"] = tokenized_data["input_ids"].copy()
        
        dataset = Dataset.from_dict(tokenized_data)
        
        # Print an example from the dataset
        print("\nDataset example:")
        print(dataset[0])
        
        return dataset

    print("Training data path:")
    print(os.path.join(args.train_dir, 'exaone_train_set.tsv'))

    train_dataset = load_and_tokenize_data(os.path.join(args.train_dir, 'exaone_train_set.tsv'))
    eval_dataset = load_and_tokenize_data(os.path.join(args.validation_dir, 'exaone_validation_set.tsv'))

    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )
    
    # Define separate directory for checkpoints
    checkpoints_dir = os.path.join(args.model_dir, "checkpoints")
    os.makedirs(checkpoints_dir, exist_ok=True)

    # Training arguments with DeepSpeed integration
    training_args = TrainingArguments(
        output_dir=checkpoints_dir,  # Checkpoints saved here,
        num_train_epochs=args.epochs,
        per_device_train_batch_size=args.batch_size,
        per_device_eval_batch_size=args.batch_size,
        gradient_accumulation_steps=args.gradient_accumulation_steps,
        learning_rate=args.learning_rate,
        weight_decay=3e-7,
        bf16=True,
        logging_dir=os.path.join(args.output_data_dir, "logs"),
        logging_steps=10,
        log_level='debug',
        #eval_steps=50,
        #save_total_limit=1,
        load_best_model_at_end=False,
        evaluation_strategy="epoch",         # Match with save_strategy
        save_strategy="epoch",               # Ensure it matches evaluation_strategy
        save_total_limit=2,                  # Retain only best and last checkpoints
        metric_for_best_model="eval_loss",    # Specify the metric for best model
        #metric_for_best_model="eval_loss",
        dataloader_num_workers=4,
        deepspeed=deepspeed_config#"deepspeed_config.json",  # Specify DeepSpeed config file
    )

    # Custom callback to log GPU stats
    class GPUStatsCallback(TrainerCallback):
        def on_step_end(self, args, state, control, **kwargs):
            gpu = int(os.environ.get("LOCAL_RANK", -1))
            if gpu == -1:
                return  # Skip if not using GPU
            # Synchronize to ensure all computations are done
            torch.cuda.synchronize(gpu)
            allocated = torch.cuda.memory_allocated(gpu) / 1e9  # Convert to GB
            reserved = torch.cuda.memory_reserved(gpu) / 1e9  # Convert to GB

            # Optionally, get GPU utilization using nvidia-smi
            try:
                result = subprocess.check_output(
                    ['nvidia-smi', '--id={}'.format(gpu), '--query-gpu=utilization.gpu,memory.used,memory.total', '--format=csv,nounits,noheader'],
                    encoding='utf-8'
                )
                gpu_util, mem_used, mem_total = result.strip().split(',')
                gpu_util = int(gpu_util)
                mem_used = float(mem_used) / 1e3  # Convert MB to GB
                mem_total = float(mem_total) / 1e3  # Convert MB to GB
            except Exception as e:
                gpu_util = 'N/A'
                mem_used = allocated
                mem_total = reserved
                print(f"Error getting GPU utilization: {e}")

            print(f"After step {state.global_step}: GPU {gpu}, Utilization: {gpu_util}%, Memory Used: {mem_used:.2f} GB / {mem_total:.2f} GB")
            
    # Initialize Trainer with DeepSpeed and callbacks
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        #processing_class=tokenizer,
        data_collator=data_collator,
        callbacks=[GPUStatsCallback()],  # Add the custom callback here
    )

    # Start training
    trainer.train()
    
    
    # Save the model - **all processes must call this**
    print("Model trained successfully, proceeding to save...")
    trainer.save_model(args.model_dir)  # All processes call this

    # Only the main process handles tokenizer saving, model card creation, and cleanup
    if trainer.is_world_process_zero():
        print("Saving tokenizer and creating model card...")
        tokenizer.save_pretrained(args.model_dir)
        trainer.create_model_card()

        print("Saving completed. Verifying saved files...")
        # List the contents of the model directory
        list_dir_contents(args.model_dir)

        # Optionally, remove the checkpoints directory to free up space
        try:
            shutil.rmtree(checkpoints_dir)
            print(f"Removed checkpoints directory: {checkpoints_dir}")
        except Exception as e:
            print(f"Error removing checkpoints directory: {e}")
        
        list_dir_contents(args.model_dir)

        print("Done!")


    """print("Model trained successfully, proceeding to save...")
    print("Saving tokenizer...")
    # Save our tokenizer and create model card
    tokenizer.save_pretrained(args.model_dir)
    trainer.create_model_card()
    # Push the results to the hub
    #if args.repository_id:
    #    trainer.push_to_hub()

    # Saves the model to s3 uses os.environ["SM_MODEL_DIR"] to make sure checkpointing works
    print("Saving model...")
    trainer.save_model(os.environ["SM_MODEL_DIR"])
    tokenizer.save_pretrained(os.environ["SM_MODEL_DIR"])
    print("done!")"""
    
    """# Only the main process should handle saving
    if trainer.is_world_process_zero():
        print("Model trained successfully, proceeding to save...")
        print("Saving tokenizer...")
        tokenizer.save_pretrained(args.model_dir)
        trainer.create_model_card()
        
        print("Saving model...")
        trainer.save_model(args.model_dir)
        tokenizer.save_pretrained(args.model_dir)
        print("done!")"""
    
    # Save the model (only on the main process)
    #print("Model trained successfully, proceeding to save...")
    #if trainer.is_world_process_zero():
    #    print("Saving ongoing...")
    #    print("Files in model_dir:", os.listdir(args.model_dir))
    #    model.save_pretrained(args.model_dir, safe_serialization=False)
    #    #trainer.save_model(args.model_dir)
    #    print("Files in model_dir:", os.listdir(args.model_dir))
    #    print("done!")

if __name__ == "__main__":
    main()


Overwriting train_deploy_huggingface.py


**4-Create the requirements file**

Since we are inside of a docker container and cannot do many things, we need to install the necessary libraries through a single command therefore all the necessary libraries are added in this requirements.txt

In [5]:
%%writefile requirements.txt

transformers
torch
datasets
accelerate
sentencepiece
bitsandbytes
peft
pyarrow
deepspeed==0.15.4
accelerate>=0.26.0

Overwriting requirements.txt


**4-Deepspeed Configuration**

Since modeldataparallel and dataparallel are not enough to parallelizde this big models, we use deepspeed. Specifically Deepspeed ZeRO Stage 3. This allows us the maximum parallelization inside of the p4d and p4de clusters.

**Important Note:** There are a few important things to take into account. Since deepspeed configuration is tied to the huggingface trainingArguments, make sure both definitions match (the one on step 3 and this one). If you are using p4d make sure that on the "device" definition is set to "cpu". You need to offload parameters and optimizers to cpu or your cluster will crash. If you have a p4de cluster, set it to "none" and the training time would be approximately half (1h vs 30mins for 7~8B family models).

In [6]:
%%writefile deepspeed_config.json
{
  "train_micro_batch_size_per_gpu": 1,
  "gradient_accumulation_steps": 4,
  "steps_per_print": 100,
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 1e-5,
      "betas": [0.9, 0.999],
      "eps": 1e-8,
      "weight_decay": 3e-7
    }
  },
  "bf16": {
    "enabled": true
  },
  "zero_optimization": {
    "stage": 3,
    "stage3_gather_16bit_weights_on_model_save": true,
    "offload_optimizer": {
      "device": "none",
      "pin_memory": true
    },
    "offload_param": {
      "device": "none",
      "pin_memory": true
    },
    "allgather_partitions": true,
    "allgather_bucket_size": 50000000,
    "overlap_comm": true,
    "reduce_scatter": true,
    "reduce_bucket_size": 50000000,
    "contiguous_gradients": true
  },
  "activation_checkpointing": {
    "partition_activations": true,
    "contiguous_memory_optimization": true
  },
  "wall_clock_breakdown": false
}




Overwriting deepspeed_config.json


**5-Deepspeed Launcher**

Unluckily we cannot launch deepspeed inside of our training job of step 3. Due to AWS and container related issues, if you try to do so, you wont be able to see the rest of the GPUs since your code is already running. In order to arrange the previous code, a deepspeed launcher must be made that directly from command line creates all the required processes (in this case 8 processes since we have 8 GPUs) and everything is coordinated by this code.

In [7]:
%%writefile ds_launcher.py
import sys
import os
import subprocess
import json
import sys
import logging
from argparse import ArgumentParser

logger = logging.getLogger(__name__)


def parse_args():
    parser = ArgumentParser(
        description=("SageMaker DeepSpeed Launch helper utility that will spawn deepspeed training scripts")
    )

    # rest from the training program
    parsed, nargs = parser.parse_known_args()

    return nargs


def main():
    # https://github.com/microsoft/DeepSpeed/blob/master/deepspeed/launcher/launch.py
    num_gpus = int(os.environ.get("SM_NUM_GPUS", 0))
    hosts = json.loads(os.environ.get("SM_HOSTS", "{}"))
    num_nodes = len(hosts)
    current_host = os.environ.get("SM_CURRENT_HOST", 0)
    rank = hosts.index(current_host)
    print(f"num_gpus = {num_gpus}, num_nodes = {num_nodes}, current_host = {current_host}, rank = {rank}")

    # os.environ['NCCL_DEBUG'] = 'INFO'

    # get number of GPU
    # if num_gpus == 0:
    #     raise ValueError("No GPUs found.")

    args = parse_args()
    command = f"deepspeed --num_gpus={num_gpus} train_deploy_huggingface.py {' '.join(args)}"
    print(f"command = {command}")
    # launch deepspeed training
    deepspeed_launch(command)


def deepspeed_launch(command):
    # try:
    try:
        subprocess.run(command, shell=True)
    except Exception as e:
        logger.info(e)


if __name__ == "__main__":
    main()

Overwriting ds_launcher.py


**6-Sagemaker Training Job Creation**

This step wraps up all of our work. Basically all files are passed here in order to be able to launch the codes inside of the container.

**Important Note:** Make sure that the instance type is set to the one you need (p4d or p4de) with the exact name AWS requires such as **ml.p4de.24xlarge** or **ml.p4d.24xlarge**. Also make sure all hyperparameters match with the previously described ones.

In [8]:
from sagemaker.pytorch import PyTorch

# Replace with your actual Role ARN
role = "YOUR_AWS_KEYS"

distribution = {
    "deepspeed": {
        "enabled": True,
        "config_path": "deepspeed_config.json"
    }
}

estimator = PyTorch(
    entry_point="ds_launcher.py",
    role=role,
    source_dir=".",  # Ensure 'deepspeed_config.json' is included here
    instance_count=1,
    instance_type="ml.p4de.24xlarge",  # Instance with 8 GPUs
    framework_version="2.1.0",  # Ensure compatibility with DeepSpeed
    py_version="py310",
    dependencies=["requirements.txt"],
    hyperparameters={
        "epochs": 4,
        "batch_size": 1,
        "learning_rate": 1e-5,
        "gradient_accumulation_steps": 4,
        "max_length": 4000,  # Updated max_length to 200
    },
    distribution=distribution,
    output_path=f"s3://{bucket_name}/{s3_prefix}/model",
)

# Define S3 URIs for TSV data
train_s3_uri = f"s3://{bucket_name}/{train_s3_path}"
validation_s3_uri = f"s3://{bucket_name}/{validation_s3_path}"

# Fit the model
estimator.fit({"train": train_s3_uri, "validation": validation_s3_uri})


[01/22/25 15:16:39] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=842655;file://C:\Users\javia\anaconda3\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=956955;file://C:\Users\javia\anaconda3\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[01/22/25 15:16:40] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=747823;file://C:\Users\javia\anaconda3\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=125499;file://C:\Users\javia\anaconda3\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[01/22/25 15:16:41] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=937174;file://C:\Users\javia\anaconda3\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=966325;file://C:\Users\javia\anaconda3\lib\site-packages\sagemaker\image_uris.py#674\674]8;;\
                             instance_type, framework etc.                                                         

[01/22/25 15:18:29] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=668752;file://C:\Users\javia\anaconda3\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=696132;file://C:\Users\javia\anaconda3\lib\site-packages\sagemaker\image_uris.py#674\674]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name:                                       ]8;id=261014;file://C:\Users\javia\anaconda3\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=714583;file://C:\Users\javia\anaconda3\lib\site-packages\sagemaker\session.py#1042\1042]8;;\
                             pytorch-training-2025-01-22-06-16-41-206                                              

...

After step 173: GPU 0, Utilization: 0%, Memory Used: 46.16 GB / 81.92 GB
88%|████████▊ | 173/196 [39:52<04:45, 12.40s/it]
After step 173: GPU 1, Utilization: 6%, Memory Used: 47.70 GB / 81.92 GB
After step 173: GPU 3, Utilization: 0%, Memory Used: 47.56 GB / 81.92 GB
After step 173: GPU 6, Utilization: 63%, Memory Used: 49.41 GB / 81.92 GB
After step 173: GPU 4, Utilization: 0%, Memory Used: 47.64 GB / 81.92 GB
After step 173: GPU 2, Utilization: 22%, Memory Used: 47.58 GB / 81.92 GB
After step 173: GPU 5, Utilization: 12%, Memory Used: 47.43 GB / 81.92 GB
After step 173: GPU 7, Utilization: 0%, Memory Used: 47.58 GB / 81.92 GB
After step 174: GPU 0, Utilization: 77%, Memory Used: 46.16 GB / 81.92 GB
89%|████████▉ | 174/196 [40:04<04:32, 12.37s/it]
After step 174: GPU 1, Utilization: 100%, Memory Used: 47.70 GB / 81.92 GB
After step 174: GPU 3, Utilization: 71%, Memory Used: 47.56 GB / 81.92 GB
After step 174: GPU 4, Utilization: 100%, Memory Used: 47.64 GB / 81.92 GB
After step 174: G

After step 185: GPU 0, Utilization: 100%, Memory Used: 46.16 GB / 81.92 GB
94%|█████████▍| 185/196 [42:20<02:15, 12.36s/it]
After step 185: GPU 3, Utilization: 100%, Memory Used: 47.56 GB / 81.92 GB
After step 185: GPU 5, Utilization: 97%, Memory Used: 47.43 GB / 81.92 GB
After step 185: GPU 4, Utilization: 64%, Memory Used: 47.64 GB / 81.92 GB
After step 185: GPU 7, Utilization: 75%, Memory Used: 47.58 GB / 81.92 GB
After step 185: GPU 6, Utilization: 79%, Memory Used: 49.41 GB / 81.92 GB
After step 185: GPU 2, Utilization: 91%, Memory Used: 47.58 GB / 81.92 GB
After step 185: GPU 1, Utilization: 60%, Memory Used: 47.70 GB / 81.92 GB
After step 186: GPU 3, Utilization: 100%, Memory Used: 47.56 GB / 81.92 GB
After step 186: GPU 0, Utilization: 100%, Memory Used: 46.16 GB / 81.92 GB
95%|█████████▍| 186/196 [42:32<02:03, 12.33s/it]
After step 186: GPU 5, Utilization: 100%, Memory Used: 47.43 GB / 81.92 GB
After step 186: GPU 2, Utilization: 100%, Memory Used: 47.58 GB / 81.92 GB
After st

Saving model checkpoint to /opt/ml/model/checkpoints/checkpoint-196
Saving model checkpoint to /opt/ml/model/checkpoints/checkpoint-196
Configuration saved in /opt/ml/model/checkpoints/checkpoint-196/config.json
Configuration saved in /opt/ml/model/checkpoints/checkpoint-196/config.json
Configuration saved in /opt/ml/model/checkpoints/checkpoint-196/generation_config.json
Configuration saved in /opt/ml/model/checkpoints/checkpoint-196/generation_config.json
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 4 checkpoint shards. You can find where each parameters has been saved in the index located at /opt/ml/model/checkpoints/checkpoint-196/model.safetensors.index.json.
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 4 checkpoint shards. You can find where each parameters has been saved in the index located at /opt/ml/model/checkpoints/checkpoint-196/model.safetensors.index.json.
tokenizer config file sa

***** Running Evaluation *****
  Num examples = 297
Batch size = 1
***** Running Evaluation *****
  Num examples = 297
  Batch size = 1
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
#011- Avoid using `tokenizers` before the fork if possible
#011- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
#011- Avoid using `tokenizers` before the fork if possible
#011- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
#011- Avoid using

The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 4 checkpoint shards. You can find where each parameters has been saved in the index located at /opt/ml/model/model.safetensors.index.json.
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 4 checkpoint shards. You can find where each parameters has been saved in the index located at /opt/ml/model/model.safetensors.index.json.
tokenizer config file saved in /opt/ml/model/tokenizer_config.json
tokenizer config file saved in /opt/ml/model/tokenizer_config.json
Special tokens file saved in /opt/ml/model/special_tokens_map.json
Special tokens file saved in /opt/ml/model/special_tokens_map.json
Saving tokenizer and creating model card...
tokenizer config file saved in /opt/ml/model/tokenizer_config.json
tokenizer config file saved in /opt/ml/model/tokenizer_config.json
Special tokens file saved in /opt/ml/model/special_tokens_map.json
Special tokens file saved i